In [5]:
import pybaseball
import pandas as pd

In [21]:
data = pybaseball.standings(2012)
data

[                  Tm   W   L  W-L%    GB
 1   New York Yankees  95  67  .586    --
 2  Baltimore Orioles  93  69  .574   2.0
 3     Tampa Bay Rays  90  72  .556   5.0
 4  Toronto Blue Jays  73  89  .451  22.0
 5     Boston Red Sox  69  93  .426  26.0,
                    Tm   W   L  W-L%    GB
 1      Detroit Tigers  88  74  .543    --
 2   Chicago White Sox  85  77  .525   3.0
 3  Kansas City Royals  72  90  .444  16.0
 4   Cleveland Indians  68  94  .420  20.0
 5     Minnesota Twins  66  96  .407  22.0,
                               Tm   W   L  W-L%    GB
 1              Oakland Athletics  94  68  .580    --
 2                  Texas Rangers  93  69  .574   1.0
 3  Los Angeles Angels of Anaheim  89  73  .549   5.0
 4               Seattle Mariners  75  87  .463  19.0,
                       Tm   W   L  W-L%    GB
 1   Washington Nationals  98  64  .605    --
 2         Atlanta Braves  94  68  .580   4.0
 3  Philadelphia Phillies  81  81  .500  17.0
 4          New York Mets  74  88

## We can observe that the team with the best record in 2012 were the Nationals with a 98-64 record

* Let's plot a regression equation for Wins and Run Differential until 2012

In [22]:
df = pd.read_csv("baseball.csv")

In [23]:
from sklearn.linear_model import LinearRegression
from sklearn import metrics

lr = LinearRegression()

y = df["W"]
df["RD"] = df["RS"]

,Team,League,Year,RS,RA,W,OBP,SLG,BA,Playoffs,RankSeason,RankPlayoffs,G,OOBP,OSLG
0,ARI,NL,2012,734,688,81,0.328,0.418,0.259,0,NaN,NaN,162,0.317,0.415
1,ATL,NL,2012,700,600,94,0.320,0.389,0.247,1,4.0,5.0,162,0.306,0.378
2,BAL,AL,2012,712,705,93,0.311,0.417,0.247,1,5.0,4.0,162,0.315,0.403
3,BOS,AL,2012,734,806,69,0.315,0.415,0.260,0,NaN,NaN,162,0.331,0.428
4,CHC,NL,2012,613,759,61,0.302,0.378,0.240,0,NaN,NaN,162,0.335,0.424
5,CHW,AL,2012,748,676,85,0.318,0.422,0.255,0,NaN,NaN,162,0.319,0.405
6,CIN,NL,2012,669,588,97,0.315,0.411,0.251,1,2.0,4.0,162,0.305,0.390
7,CLE,AL,2012,667,845,68,0.324,0.381,0.251,0,NaN,NaN,162,0.336,0.430
8,COL,NL,2012,758,890,64,0.330,0.436,0.274,0,NaN,NaN,162,0.357,0.470
9,DET,AL,2012,726,670,88,0.335,0.422,0.268,1,6.0,2.0,162,0.314,0.402
